In [1]:
# Core Python + plotting
import numpy as np
import matplotlib.pyplot as plt

# Qiskit core
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.visualization import plot_histogram

# Aer simulator
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, pauli_error, depolarizing_error, amplitude_damping_error

import warnings
warnings.filterwarnings("ignore")

print("Libraries imported successfully!")

/var/folders/x6/c44jk2f54jl2th8706q9sv500000gn/T/ipykernel_2276/3316037889.py:6: DeprecationWarning: Using Qiskit with Python 3.9 is deprecated as of the 2.1.0 release. Support for running Qiskit with Python 3.9 will be removed in the 2.3.0 release, which coincides with when Python 3.9 goes end of life.
  from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister


Libraries imported successfully!


In [2]:
# Bit-flip (X) noise on id
def get_bitflip_noise(p_noise):
    if p_noise == 0:
        return None
    error = pauli_error([("X", p_noise), ("I", 1 - p_noise)])
    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(error, "id")
    return noise_model

# Depolarizing noise on id (1-qubit)
def get_depolarizing_noise(p_noise):
    if p_noise == 0:
        return None
    error = depolarizing_error(p_noise, 1)
    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(error, "id")
    return noise_model

# Amplitude damping noise on id
def get_amplitude_damping_noise(p_noise):
    if p_noise == 0:
        return None
    error = amplitude_damping_error(p_noise)
    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(error, "id")
    return noise_model

# Bit-phase flip (Y) noise on id
def get_bitphaseflip_noise(p_noise):
    if p_noise == 0:
        return None
    error = pauli_error([("Y", p_noise), ("I", 1 - p_noise)])
    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(error, "id")
    return noise_model

print("Noise model functions defined!")

Noise model functions defined!


In [3]:
# Single-qubit standard noisy channel circuit
def build_standard_noisy_circuit(initial_state):
    qr = QuantumRegister(1, "q")
    cr = ClassicalRegister(1, "c")
    qc = QuantumCircuit(qr, cr)

    if initial_state == 1:
        qc.x(0)
    qc.barrier(label="Start")

    qc.id(0)  # single noisy segment

    qc.barrier(label="After Channel")
    qc.measure(0, 0)
    return qc

print("Standard circuit builder defined!")


# QZE: N segments, measurement only at end (noise split across segments)
def build_qze_simple_circuit(initial_state, n_steps):
    qr = QuantumRegister(1, "q")
    cr = ClassicalRegister(1, "c")
    qc = QuantumCircuit(qr, cr)

    if initial_state == 1:
        qc.x(0)
    qc.barrier(label="Start")

    for i in range(n_steps):
        qc.id(0)
        if i < n_steps - 1:
            qc.barrier(label=f"Step {i+1}")

    qc.barrier(label="End")
    qc.measure(0, 0)
    return qc

print("QZE circuit builder defined!")


Standard circuit builder defined!
QZE circuit builder defined!


In [4]:
N_QUBITS = 200
SHOTS_PER_QUBIT = 1

def get_noise_model(p_noise, noise_type):
    if noise_type == "bitflip":
        return get_bitflip_noise(p_noise)
    elif noise_type == "depolarizing":
        return get_depolarizing_noise(p_noise)
    elif noise_type == "amplitude_damping":
        return get_amplitude_damping_noise(p_noise)
    elif noise_type == "bitphaseflip":
        return get_bitphaseflip_noise(p_noise)
    else:
        raise ValueError("Unknown noise_type")

def run_standard_channel(n_qubits, initial_state, p_noise, noise_type):
    noise_model = get_noise_model(p_noise, noise_type)
    simulator = AerSimulator(noise_model=noise_model) if noise_model else AerSimulator()

    errors = 0
    for _ in range(n_qubits):
        qc = build_standard_noisy_circuit(initial_state)
        job = simulator.run(qc, shots=SHOTS_PER_QUBIT, memory=True)
        measured_bit = int(job.result().get_memory()[0])
        if measured_bit != initial_state:
            errors += 1

    return errors / n_qubits

print("Standard channel simulation function defined!")

def run_qze_channel(n_qubits, initial_state, p_noise, n_steps, noise_type):
    if n_steps <= 0:
        raise ValueError("n_steps must be >= 1")

    # simple split: p_step = p / N (same as your previous notebook)[file:51][file:71]
    p_step = p_noise / max(1, n_steps)
    noise_model = get_noise_model(p_step, noise_type)
    simulator = AerSimulator(noise_model=noise_model) if noise_model else AerSimulator()

    errors = 0
    for _ in range(n_qubits):
        qc = build_qze_simple_circuit(initial_state, n_steps)
        job = simulator.run(qc, shots=SHOTS_PER_QUBIT, memory=True)
        measured_bit = int(job.result().get_memory()[0])
        if measured_bit != initial_state:
            errors += 1

    return errors / n_qubits

print("QZE channel simulation function defined!")


Standard channel simulation function defined!
QZE channel simulation function defined!


In [5]:
from qiskit import QuantumCircuit

def fold_global(circuit, scale_factor):
    """
    Global folding: odd integer scale_factor = 1,3,5,...
    U -> U (U† U)^k, measurements re-attached at end.[web:55][web:67]
    """
    if scale_factor < 1:
        raise ValueError("scale_factor must be >= 1")
    n = int(round(scale_factor))
    if n % 2 == 0:
        n += 1

    # Copy and remove measurement to define U
    base = circuit.copy()
    base.remove_final_measurements()

    inv = base.inverse()

    body = QuantumCircuit(base.num_qubits, base.num_clbits)
    body.compose(base, inplace=True)
    k = (n - 1) // 2
    for _ in range(k):
        body.compose(inv, inplace=True)
        body.compose(base, inplace=True)

    folded = QuantumCircuit(base.num_qubits, circuit.num_clbits)
    folded.compose(body, inplace=True)

    # robust index mapping
    q_index = {q: i for i, q in enumerate(circuit.qubits)}
    c_index = {c: i for i, c in enumerate(circuit.clbits)}

    for instr, qargs, cargs in circuit.data:
        if instr.name == "measure":
            q = q_index[qargs[0]]
            c = c_index[cargs[0]]
            folded.measure(q, c)

    return folded


In [6]:
def zne_ber(
    initial_state,
    p_noise,
    noise_type,
    scale_factors=(1, 3, 5),
    n_qubits=N_QUBITS,
    n_shots=1,
):
    noise_model = get_noise_model(p_noise, noise_type)
    simulator = AerSimulator(noise_model=noise_model) if noise_model else AerSimulator()

    bers = []
    s_vals = []

    for s in scale_factors:
        qc_base = build_standard_noisy_circuit(initial_state)
        qc_folded = fold_global(qc_base, s)

        errors = 0
        for _ in range(n_qubits):
            job = simulator.run(qc_folded, shots=n_shots, memory=True)
            bit = int(job.result().get_memory()[0])
            if bit != initial_state:
                errors += 1
        ber_s = errors / n_qubits
        bers.append(ber_s)
        s_vals.append(s)

    s_vals = np.array(s_vals, dtype=float)
    bers = np.array(bers, dtype=float)

    # Linear fit BER(s) = a + b*s, extrapolate a = BER(0)[web:55][web:67]
    coeffs = np.polyfit(s_vals, bers, 1)
    a = coeffs[1]
    ber_0 = max(0.0, min(1.0, a))
    return ber_0, (s_vals, bers)


In [7]:
from qiskit.circuit.library import XGate, YGate

X = XGate()
Y = YGate()
dd_sequence = [X, Y, X, Y]  # XY4

def build_dd_manual_circuit(initial_state, n_segments=10):
    """
    DD: for each noisy id segment, add XY4 pulses around it.
    Same total number of channel segments as ZNE/QZE for fairness.[web:43][web:65]
    """
    qr = QuantumRegister(1, "q")
    cr = ClassicalRegister(1, "c")
    qc = QuantumCircuit(qr, cr)

    if initial_state == 1:
        qc.x(0)
    qc.barrier(label="Start")

    for _ in range(n_segments):
        qc.id(0)
        qc.x(0)
        qc.y(0)
        qc.x(0)
        qc.y(0)

    qc.barrier(label="End")
    qc.measure(0, 0)
    return qc

def dd_ber(
    initial_state,
    p_noise,
    noise_type,
    n_segments=10,
    n_qubits=N_QUBITS,
    n_shots=1,
):
    noise_model = get_noise_model(p_noise, noise_type)
    simulator = AerSimulator(noise_model=noise_model) if noise_model else AerSimulator()

    qc_dd = build_dd_manual_circuit(initial_state, n_segments=n_segments)

    errors = 0
    for _ in range(n_qubits):
        job = simulator.run(qc_dd, shots=n_shots, memory=True)
        bit = int(job.result().get_memory()[0])
        if bit != initial_state:
            errors += 1

    return errors / n_qubits, qc_dd


In [8]:
def pec_sample_bitflip_like(
    initial_state,
    p_noise,
    n_qubits=N_QUBITS,
    n_shots=1,
):
    """
    Toy PEC for 1-qubit Pauli channel with dominant X (bit-flip-like).
    For bit-flip / depolarizing, gives a reasonable illustration.[file:2][web:60][web:69]
    """
    if p_noise == 0:
        return 0.0

    denom = 1 - 2 * p_noise
    if abs(denom) < 1e-6:
        # extremely noisy; fall back to standard
        return None

    q_I = 1 / denom
    q_X = -p_noise / (denom * (1 - p_noise))
    gamma = abs(q_I) + abs(q_X)

    noise_model = get_bitflip_noise(p_noise)
    simulator = AerSimulator(noise_model=noise_model)

    qr = QuantumRegister(1, "q")
    cr = ClassicalRegister(1, "c")

    weighted_sum_wrong = 0.0
    total_weight = 0.0

    for _ in range(n_qubits):
        if np.random.rand() < abs(q_I) / gamma:
            inv_op = "I"
            w = np.sign(q_I) * gamma
        else:
            inv_op = "X"
            w = np.sign(q_X) * gamma

        qc = QuantumCircuit(qr, cr)
        if initial_state == 1:
            qc.x(0)
        qc.barrier(label="Start")

        qc.id(0)

        if inv_op == "X":
            qc.x(0)

        qc.barrier(label="End")
        qc.measure(0, 0)

        job = simulator.run(qc, shots=n_shots, memory=True)
        bit = int(job.result().get_memory()[0])

        if bit != initial_state:
            weighted_sum_wrong += w
        total_weight += abs(w)

    ber_hat = weighted_sum_wrong / total_weight
    return max(0.0, min(1.0, ber_hat))


In [9]:
def count_gates(qc):
    return sum(1 for instr, _, _ in qc.data if instr.name not in ["barrier"])

def overhead_factor(gates_method, gates_standard):
    if gates_standard == 0:
        return 1.0
    return gates_method / gates_standard


In [10]:
initial_state = 1
p_noise = 0.25
N_QZE_STEPS = 10
N_SEGMENTS_ZNE_DD = 10
TRIALS = 3  # repeat to average randomness

noise_types = ["bitflip", "depolarizing", "amplitude_damping", "bitphaseflip"]
methods = ["standard", "qze", "zne", "dd", "pec"]

results_ber = {nt: {m: None for m in methods} for nt in noise_types}
results_overhead = {nt: {m: None for m in methods} for nt in noise_types}

print(f"Running BER and overhead comparison at p = {p_noise}...")

for noise_type in noise_types:
    # Standard
    std_vals = []
    for _ in range(TRIALS):
        std_vals.append(run_standard_channel(N_QUBITS, initial_state, p_noise, noise_type))
    ber_std = np.mean(std_vals)
    results_ber[noise_type]["standard"] = ber_std

    qc_std = build_standard_noisy_circuit(initial_state)
    gates_std = count_gates(qc_std)
    results_overhead[noise_type]["standard"] = 1.0  # by definition

    # QZE
    qze_vals = []
    for _ in range(TRIALS):
        qze_vals.append(run_qze_channel(N_QUBITS, initial_state, p_noise, N_QZE_STEPS, noise_type))
    ber_qze = np.mean(qze_vals)
    results_ber[noise_type]["qze"] = ber_qze

    qc_qze = build_qze_simple_circuit(initial_state, N_QZE_STEPS)
    gates_qze = count_gates(qc_qze)
    results_overhead[noise_type]["qze"] = overhead_factor(gates_qze, gates_std)

    # ZNE
    zne_val, _ = zne_ber(
        initial_state,
        p_noise,
        noise_type,
        scale_factors=(1, 3, 5),
        n_qubits=N_QUBITS,
        n_shots=1,
    )
    results_ber[noise_type]["zne"] = zne_val

    # Use folded circuit at largest scale for gate count
    qc_zne = fold_global(qc_std, 5)
    gates_zne = count_gates(qc_zne)
    results_overhead[noise_type]["zne"] = overhead_factor(gates_zne, gates_std)

    # DD
    dd_vals = []
    for _ in range(TRIALS):
        dd_ber_val, qc_dd = dd_ber(
            initial_state,
            p_noise,
            noise_type,
            n_segments=N_SEGMENTS_ZNE_DD,
            n_qubits=N_QUBITS,
            n_shots=1,
        )
        dd_vals.append(dd_ber_val)
    ber_dd = np.mean(dd_vals)
    results_ber[noise_type]["dd"] = ber_dd

    gates_dd = count_gates(qc_dd)
    results_overhead[noise_type]["dd"] = overhead_factor(gates_dd, gates_std)

    # PEC (only meaningful for bitflip-like; else mark None)
    if noise_type in ["bitflip", "depolarizing"]:
        pec_vals = []
        for _ in range(TRIALS):
            ber_pec = pec_sample_bitflip_like(initial_state, p_noise, n_qubits=N_QUBITS, n_shots=1)
            pec_vals.append(ber_pec)
        ber_pec_mean = np.mean(pec_vals)
        results_ber[noise_type]["pec"] = ber_pec_mean

        # approximate gate count ~ standard + one possible X
        qc_pec = qc_std.copy()
        qc_pec.x(0)
        gates_pec = count_gates(qc_pec)
        results_overhead[noise_type]["pec"] = overhead_factor(gates_pec, gates_std)
    else:
        results_ber[noise_type]["pec"] = None
        results_overhead[noise_type]["pec"] = None

print("Done.")


Running BER and overhead comparison at p = 0.25...
Done.


In [11]:
print(f"BER at p = {p_noise}")
print("| Noise type        | Method    | BER       |")
print("|-------------------|-----------|-----------|")

for nt in noise_types:
    for m in methods:
        ber = results_ber[nt][m]
        if ber is None:
            ber_str = "N/A"
        else:
            ber_str = f"{ber:.4f}"
        print(f"| {nt:17s} | {m:9s} | {ber_str:9s} |")


BER at p = 0.25
| Noise type        | Method    | BER       |
|-------------------|-----------|-----------|
| bitflip           | standard  | 0.2533    |
| bitflip           | qze       | 0.1783    |
| bitflip           | zne       | 0.2212    |
| bitflip           | dd        | 0.4933    |
| bitflip           | pec       | 0.0217    |
| depolarizing      | standard  | 0.1233    |
| depolarizing      | qze       | 0.1133    |
| depolarizing      | zne       | 0.1062    |
| depolarizing      | dd        | 0.4750    |
| depolarizing      | pec       | 0.0133    |
| amplitude_damping | standard  | 0.2283    |
| amplitude_damping | qze       | 0.2267    |
| amplitude_damping | zne       | 0.1467    |
| amplitude_damping | dd        | 0.9367    |
| amplitude_damping | pec       | N/A       |
| bitphaseflip      | standard  | 0.2517    |
| bitphaseflip      | qze       | 0.2183    |
| bitphaseflip      | zne       | 0.2221    |
| bitphaseflip      | dd        | 0.5433    |
| bitphaseflip    

In [12]:
print(f"Gate-count overhead factor at p = {p_noise}")
print("| Noise type        | Method    | Overhead  |")
print("|-------------------|-----------|-----------|")

for nt in noise_types:
    for m in methods:
        ov = results_overhead[nt][m]
        if ov is None:
            ov_str = "N/A"
        else:
            ov_str = f"{ov:.2f}"
        print(f"| {nt:17s} | {m:9s} | {ov_str:9s} |")


Gate-count overhead factor at p = 0.25
| Noise type        | Method    | Overhead  |
|-------------------|-----------|-----------|
| bitflip           | standard  | 1.00      |
| bitflip           | qze       | 4.00      |
| bitflip           | zne       | 3.67      |
| bitflip           | dd        | 17.33     |
| bitflip           | pec       | 1.33      |
| depolarizing      | standard  | 1.00      |
| depolarizing      | qze       | 4.00      |
| depolarizing      | zne       | 3.67      |
| depolarizing      | dd        | 17.33     |
| depolarizing      | pec       | 1.33      |
| amplitude_damping | standard  | 1.00      |
| amplitude_damping | qze       | 4.00      |
| amplitude_damping | zne       | 3.67      |
| amplitude_damping | dd        | 17.33     |
| amplitude_damping | pec       | N/A       |
| bitphaseflip      | standard  | 1.00      |
| bitphaseflip      | qze       | 4.00      |
| bitphaseflip      | zne       | 3.67      |
| bitphaseflip      | dd        | 17.33  

In [13]:
# Compute percentage improvement in BER of each method vs. standard for all noise types

improvement_ber = {nt: {} for nt in noise_types}

for nt in noise_types:
    ber_std = results_ber[nt]["standard"]
    for m in methods:
        ber_m = results_ber[nt][m]
        if ber_m is None or ber_std == 0:
            improvement_ber[nt][m] = None
        else:
            # positive value means BER reduced relative to standard (in percent)
            improvement_ber[nt][m] = 100.0 * (ber_std - ber_m) / ber_std

print("Percentage BER improvement relative to standard (positive = better):")
print("| Noise type        | Method     | Improvement (%) |")
print("|-------------------|-----------|-----------------|")
for nt in noise_types:
    for m in methods:
        val = improvement_ber[nt][m]
        if val is None:
            s = "N/A"
        else:
            s = f"{val:.1f}"
        print(f"| {nt:17s} | {m:9s} | {s:15s} |")


Percentage BER improvement relative to standard (positive = better):
| Noise type        | Method     | Improvement (%) |
|-------------------|-----------|-----------------|
| bitflip           | standard  | 0.0             |
| bitflip           | qze       | 29.6            |
| bitflip           | zne       | 12.7            |
| bitflip           | dd        | -94.7           |
| bitflip           | pec       | 91.4            |
| depolarizing      | standard  | 0.0             |
| depolarizing      | qze       | 8.1             |
| depolarizing      | zne       | 13.9            |
| depolarizing      | dd        | -285.1          |
| depolarizing      | pec       | 89.2            |
| amplitude_damping | standard  | 0.0             |
| amplitude_damping | qze       | 0.7             |
| amplitude_damping | zne       | 35.8            |
| amplitude_damping | dd        | -310.2          |
| amplitude_damping | pec       | N/A             |
| bitphaseflip      | standard  | 0.0         